In [18]:
import sympy as sp
h = sp.symbols('hbar')

# Pauli & unidad
sx = sp.Matrix([[0,1],[1,0]])
sy = sp.Matrix([[0,-sp.I],[sp.I,0]])
sz = sp.Matrix([[1,0],[0,-1]])
I  = sp.eye(2)

# Espín ½
Sx, Sy, Sz = [3*h/2*m for m in (sx, sy, sz)]

# Operadores de los dos espines
def kprod(A,B): return sp.kronecker_product(A,B)
Sx1, Sy1, Sz1 = [kprod(op,I) for op in (Sx,Sy,Sz)]
Sx2, Sy2, Sz2 = [kprod(I,op) for op in (Sx,Sy,Sz)]

# Espín total y S^2
Sx_tot = Sx1 + Sx2
Sy_tot = Sy1 + Sy2
Sz_tot = Sz1 + Sz2
S2  = sp.simplify(Sx_tot**2 + Sy_tot**2 + Sz_tot**2)

Sx_tot

Matrix([
[       0, 3*hbar/2, 3*hbar/2,        0],
[3*hbar/2,        0,        0, 3*hbar/2],
[3*hbar/2,        0,        0, 3*hbar/2],
[       0, 3*hbar/2, 3*hbar/2,        0]])

In [19]:
Sy_tot

Matrix([
[         0, -3*I*hbar/2, -3*I*hbar/2,           0],
[3*I*hbar/2,           0,           0, -3*I*hbar/2],
[3*I*hbar/2,           0,           0, -3*I*hbar/2],
[         0,  3*I*hbar/2,  3*I*hbar/2,           0]])

In [20]:
S2

Matrix([
[18*hbar**2,         0,         0,          0],
[         0, 9*hbar**2, 9*hbar**2,          0],
[         0, 9*hbar**2, 9*hbar**2,          0],
[         0,         0,         0, 18*hbar**2]])

In [21]:
eig_data = S2.eigenvects()   # [(valor propio, multiplicidad, [vectores])]
eig_data

[(0,
  1,
  [Matrix([
   [ 0],
   [-1],
   [ 1],
   [ 0]])]),
 (18*hbar**2,
  3,
  [Matrix([
   [1],
   [0],
   [0],
   [0]]),
   Matrix([
   [0],
   [1],
   [1],
   [0]]),
   Matrix([
   [0],
   [0],
   [0],
   [1]])])]

In [22]:
def norm(v):
    v = sp.Matrix(v)
    return v / sp.sqrt((v.H*v)[0])

states = []   # [(λ_s2, m, vector)]
for eigval, mult, vecs in S2.eigenvects():
    for vec in vecs:
        v = norm(vec)
        # m = valor propio de S_z / ℏ  (porque son simultáneamente propios)
        m = sp.simplify((v.H*Sz_tot*v)[0] / (h*(v.H*v)[0]))
        states.append((eigval, m, v))

# Orden: λ=2ℏ² primero (triplete), dentro de él m = +1,0,-1; luego λ=0 (singlete)
states = sorted(states,
                key=lambda t: (-float(t[0].subs(h,1)),   # S² descendente
                               -float(t[1])))            # m   descendente

# Matriz U: columnas = vectores ordenados
U = sp.Matrix.hstack(*[v for _,_,v in states])
U

Matrix([
[1,         0, 0,          0],
[0, sqrt(2)/2, 0, -sqrt(2)/2],
[0, sqrt(2)/2, 0,  sqrt(2)/2],
[0,         0, 1,          0]])

In [23]:
(U.H * S2 * U).simplify()

Matrix([
[18*hbar**2,          0,          0, 0],
[         0, 18*hbar**2,          0, 0],
[         0,          0, 18*hbar**2, 0],
[         0,          0,          0, 0]])

In [24]:
(U.H * Sz_tot * U).simplify()

Matrix([
[3*hbar, 0,       0, 0],
[     0, 0,       0, 0],
[     0, 0, -3*hbar, 0],
[     0, 0,       0, 0]])

In [25]:
U.H*U

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])